In [23]:
!py -3.10 -m venv myvenv
!myvenv\Scripts\activate

!pip install imutils
!pip install roboflow
!pip install pytube --upgrade

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import torch
torch.__version__

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
!pip install super-gradients
from super_gradients.training import models

yolo_nas_l = models.get("yolo_nas_l", pretrained_weights="coco")

In [ ]:
url = "https://previews.123rf.com/images/freeograph/freeograph2011/freeograph201100150/158301822-group-of-friends-gathering-around-table-at-home.jpg"
yolo_nas_l.predict(url, conf=0.25).show()

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name='fall_detection_1.mp4', length=len(uploaded['fall_detection_3.mp4'])))

In [ ]:
input_video_path = "fall_detection_4.mp4"
output_video_path = "detections.mp4"
device=0

yolo_nas_l.to(device).predict(input_video_path).save(output_video_path)

In [ ]:
from super_gradients.training import Trainer

CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='my_first_yolonas_run', ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

In [ ]:
dataset_params = {
    'data_dir':'drive/MyDrive/FallDetection1',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'val/images',
    'val_labels_dir':'val/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': ['Fall-Detected','Walking', 'Sitting']
}

In [ ]:
from IPython.display import clear_output

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)
test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

In [ ]:
train_data.dataset.transforms

In [ ]:
# train_data.dataset.dataset_params['transforms'][1]['DetectionRandomAffine']['degrees'] = 10.42

In [ ]:
train_data.dataset.plot()

In [ ]:
from super_gradients.training import models
model = models.get('yolo_nas_l',
                   num_classes=len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )

[2023-12-11 10:20:42] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-12-11 10:20:44] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 10,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

[2023-12-11 10:20:50] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20231211_102050_690038`
[2023-12-11 10:20:50] INFO - sg_trainer.py - Checkpoints directory: checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038
[2023-12-11 10:20:50] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/console_Dec11_10_20_50.txt


/usr/local/lib/python3.10/dist-packages/super_gradients/common/registry/registry.py:72: DeprecationWarning: Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
  warnings.warn(f"Object name `{name}` is now deprecated. Please replace it with `{deprecated_names[name]}`.", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/super_gradients/training/utils/optimizer_utils.py:107: DeprecationWarning: initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})
  warnings.warn(
[2023-12-11 10:20:55] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Full dataset size:

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.2309
│   ├── Ppyoloeloss/loss_iou = 0.5959
│   ├── Ppyoloeloss/loss_dfl = 0.7393
│   └── Ppyoloeloss/loss = 3.5661
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.5579
    ├── Ppyoloeloss/loss_iou = 0.3913
    ├── Ppyoloeloss/loss_dfl = 0.5877
    ├── Ppyoloeloss/loss = 3.5369
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.115
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]
[2023-12-11 10:23:08] INFO - base_sg_logger.py - Checkpoint saved in checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/ckpt_best.pth
[2023-12-11 10:23:08] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.1699979454278946


...

Validating epoch 8: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]
[2023-12-11 10:33:21] INFO - base_sg_logger.py - Checkpoint saved in checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/ckpt_best.pth
[2023-12-11 10:33:21] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.4355318248271942


SUMMARY OF EPOCH 8
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.1183
│   │   ├── Epoch N-1      = 1.1397 (↘ -0.0214)
│   │   └── Best until now = 1.1397 (↘ -0.0214)
│   ├── Ppyoloeloss/loss_iou = 0.4606
│   │   ├── Epoch N-1      = 0.4801 (↘ -0.0195)
│   │   └── Best until now = 0.4801 (↘ -0.0195)
│   ├── Ppyoloeloss/loss_dfl = 0.6476
│   │   ├── Epoch N-1      = 0.6609 (↘ -0.0133)
│   │   └── Best until now = 0.6609 (↘ -0.0133)
│   └── Ppyoloeloss/loss = 2.2265
│       ├── Epoch N-1      = 2.2807 (↘ -0.0542)
│       └── Best until now = 2.2807 (↘ -0.0542)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.3082
    │   ├── Epoch N-1      = 1.4205 (↘ -0.1124)
    │   └── Best until now = 1.4205 (↘ -0.1124)
    ├── Ppyoloeloss/loss_iou = 0.4602
    │   ├── Epoch N-1      = 0.4494 (↗ 0.0108)
    │   └── Best until now = 0.3913 (↗ 0.0689)
    ├── Ppyoloeloss/loss_dfl = 0.6083
    │   ├── Epoch N-1      = 0.6123 (↘ -0.0041)
    │   └── Best until now = 0.5877 (↗ 0.0205)
    ├── Ppyoloeloss/loss 

Validating epoch 9: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]
[2023-12-11 10:35:47] INFO - base_sg_logger.py - Checkpoint saved in checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/ckpt_best.pth
[2023-12-11 10:35:47] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.5317127704620361


SUMMARY OF EPOCH 9
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.0627
│   │   ├── Epoch N-1      = 1.1183 (↘ -0.0556)
│   │   └── Best until now = 1.1183 (↘ -0.0556)
│   ├── Ppyoloeloss/loss_iou = 0.441
│   │   ├── Epoch N-1      = 0.4606 (↘ -0.0196)
│   │   └── Best until now = 0.4606 (↘ -0.0196)
│   ├── Ppyoloeloss/loss_dfl = 0.6141
│   │   ├── Epoch N-1      = 0.6476 (↘ -0.0335)
│   │   └── Best until now = 0.6476 (↘ -0.0335)
│   └── Ppyoloeloss/loss = 2.1178
│       ├── Epoch N-1      = 2.2265 (↘ -0.1087)
│       └── Best until now = 2.2265 (↘ -0.1087)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.212
    │   ├── Epoch N-1      = 1.3082 (↘ -0.0961)
    │   └── Best until now = 1.3082 (↘ -0.0961)
    ├── Ppyoloeloss/loss_iou = 0.3849
    │   ├── Epoch N-1      = 0.4602 (↘ -0.0752)
    │   └── Best until now = 0.3913 (↘ -0.0063)
    ├── Ppyoloeloss/loss_dfl = 0.5376
    │   ├── Epoch N-1      = 0.6083 (↘ -0.0707)
    │   └── Best until now = 0.5877 (↘ -0.0501)
    ├── Ppyoloeloss/loss

[2023-12-11 10:37:19] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 10: 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


In [22]:
best_model = models.get('yolo_nas_l',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/ckpt_best.pth")
                        #checkpoint_path="checkpoints/my_first_yolonas_run/average_model.pth")

[2024-02-23 06:20:30] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: Incorrect Checkpoint path: /content/checkpoints/my_first_yolonas_run/RUN_20231211_102050_690038/ckpt_best.pth (This should be an absolute path)

In [ ]:
trainer.test(model=best_model,
            test_loader=test_data,
            test_metrics_list=DetectionMetrics_050(score_thres=0.1,
                                                   top_k_predictions=300,
                                                   num_cls=len(dataset_params['classes']),
                                                   normalize_targets=True,
                                                   post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                          nms_top_k=1000,
                                                                                                          max_predictions=300,
                                                                                                          nms_threshold=0.7)
                                                  ))

Testing: 100%|██████████| 6/6 [00:05<00:00,  1.46it/s]

{'PPYoloELoss/loss_cls': 1.0830462,
 'PPYoloELoss/loss_iou': 0.35310233,
 'PPYoloELoss/loss_dfl': 0.57690215,
 'PPYoloELoss/loss': 2.0130506,
 'Precision@0.50': 0.023813994601368904,
 'Recall@0.50': 0.9777777791023254,
 'mAP@0.50': 0.7984395027160645,
 'F1@0.50': 0.04639395698904991}

In [ ]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"

input_video_path = "fall_detection_1.mp4"
output_video_path = "detections11.mp4"
#device=0

best_model.to(device).predict(input_video_path).save(output_video_path)

Processing Video:   0%|          | 0/180 [00:00<?, ?it/s][2023-12-11 10:38:12] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
/usr/local/lib/python3.10/dist-packages/super_gradients/training/utils/visualization/utils.py:67: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap("gist_rainbow", num_classes)
Processing Video:  72%|███████▏  | 129/180 [00:03<00:01, 39.16it/s]

In [ ]:
test_image = 'fall104.jpg'
best_model.predict(test_image).show()